In [1]:
#Dependencies
import pandas as pd
import numpy as npa
import matplotlib as plt
import os
import csv

In [3]:
#reduce file is the data with the 37 questions selected
reduced_file = "OSMI_2016_small_df.csv"
df = pd.read_csv(reduced_file)
df.head(1)

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,Do you think that discussing a mental health disorder with your employer would have negative consequences?,Would you feel comfortable discussing a mental health disorder with your coworkers?,...,"If so, what condition(s) were you diagnosed with?",Have you ever sought treatment for a mental health issue from a mental health professional?,What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,No,No,I don't know,No,Maybe,...,"Anxiety Disorder (Generalized, Social, Phobia,...",0,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes


#### <span style="color:green">Consolidating Genders into 3 Categories (.replace())</span>

In [4]:
#Reduces the gender column to 3 values: Male, Female, Other

df['What is your gender?'] = df['What is your gender?'].replace({
    'male': 'Male',
    'Male ': 'Male',
    'm': 'Male',
    'man': 'Male',
    'M2F': 'Male',
    'Cis male': 'Male',
    'Male.': 'Male',
    'Male (cis)': 'Male',
    'Man': 'Male',
    'Sex is male': 'Male',
    'cis male': 'Male',
    'Dude': 'Male',
    "I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ":'Male',
    'mail': 'Male',
    'male ': 'Male',
    'Cis Male': 'Male',
    'cisdude': 'Male',
    'cis man': 'Male',
    'MALE': 'Male',
    'M': 'Male',
    'Malr': 'Male'
})

df['What is your gender?'] = df['What is your gender?'].replace({
    'F': 'Female',
    'female': 'Female',
    'F ': 'Female',
    'female ': 'Female',
    'Woman': 'Female',
    'fm': 'Female',
    'f': 'Female',
    'Cis female ': 'Female',
    'Female ': 'Female',
    'woman': 'Female',
    'Sex is Female': 'Female',
    'female/woman': 'Female',
    'Cisgender Female': 'Female',
    "fem":'Female',
    'Female (props for making this a freeform field, though)': 'Female',
    ' Female': 'Female',
    'Cis-woman': 'Female'
})

#temp variable to hold the gender column values
g = df['What is your gender?']

# changes what is not  Male or Female,  to `other`
for i in range(len(g)):
    if g[i] not in ['Male', 'Female']:
        df['What is your gender?'] = g.replace({g[i]: 'Other'})
        
        
print(f"There are only {df['What is your gender?'].nunique()} different genders now")

There are only 3 different genders now


In [5]:
df['What is your gender?'].value_counts()

Male      1056
Female     336
Other       41
Name: What is your gender?, dtype: int64

In [9]:
print('Column count: ', len(df.columns))

Column count:  37


#### <span style="color:green">short_col: dictionary with only the Question number (Eg: Q1, Q2 ...)</span>

In [10]:
# short_col is the dictionary with only the Question number (Example: Q1, Q2 ...)

short_cols = {
                    
       'Are you self-employed?': "Q1" ,
       'How many employees does your company or organization have?': "Q2",
       'Is your employer primarily a tech company/organization?': "Q3",
       'Is your primary role within your company related to tech/IT?': "Q4",
       'Does your employer provide mental health benefits as part of healthcare coverage?': "Q5",
       'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?':"Q6",
       'Does your employer offer resources to learn more about mental health concerns and options for seeking help?':"Q7",
       'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?': "Q8",
       'Do you think that discussing a mental health disorder with your employer would have negative consequences?': "Q9",
       'Would you feel comfortable discussing a mental health disorder with your coworkers?': "Q10",
       'Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?': "Q11",
       'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?': "Q12",
       'Do you know local or online resources to seek help for a mental health disorder?': "Q13",
       'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?':"Q14",
       'If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?': "Q15",
       'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?': "Q16",
       'If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?': "Q17",
       'Did you feel that your previous employers took mental health as seriously as physical health?': "Q18",
       'Do you feel that being identified as a person with a mental health issue would hurt your career?': "Q19",
       'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?': "Q20",
       'Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?':"Q21",
       'Have your observations of how another individual who discussed a mental health disorder made you less likely to reveal a mental health issue yourself in your current workplace?':"Q22" ,
    
        'Do you have a family history of mental illness?':"Q23",
       'Have you had a mental health disorder in the past?':"Q24",
       'Do you currently have a mental health disorder?':"Q25",
       'If yes, what condition(s) have you been diagnosed with?':"Q26",
       'Have you been diagnosed with a mental health condition by a medical professional?':"Q27",
       'If so, what condition(s) were you diagnosed with?':"Q28",
       'Have you ever sought treatment for a mental health issue from a mental health professional?':"Q29",
       'What is your age?':"Q30",
       'What is your gender?': "Q31",
       'What country do you live in?': "Q32",
       'What US state or territory do you live in?': "Q33",
       'What country do you work in?': "Q34",
       'What US state or territory do you work in?':"Q35",
       'Which of the following best describes your work position?':"Q36",
       'Do you work remotely?':"Q37"
                    
}

In [11]:
#create inverse dictionary with a dictionary comprehension
 #we use the inverse dictionary to display questions in our results
long_cols = {v: k for k, v in short_cols.items()}

In [11]:
# long_cols is the dictionary with the question labels as the values
# - can be used to display only the labels in a dataframe
long_cols

{'Q1': 'Are you self-employed?',
 'Q2': 'How many employees does your company or organization have?',
 'Q3': 'Is your employer primarily a tech company/organization?',
 'Q4': 'Is your primary role within your company related to tech/IT?',
 'Q5': 'Does your employer provide mental health benefits as part of healthcare coverage?',
 'Q6': 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
 'Q7': 'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
 'Q8': 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
 'Q9': 'Do you think that discussing a mental health disorder with your employer would have negative consequences?',
 'Q10': 'Would you feel comfortable discussing a mental health disorder with your coworkers?',
 'Q11': 'Would you feel comfortable discu

In [12]:
df.columns[:5]

Index(['Are you self-employed?',
       'How many employees does your company or organization have?',
       'Is your employer primarily a tech company/organization?',
       'Is your primary role within your company related to tech/IT?',
       'Does your employer provide mental health benefits as part of healthcare coverage?'],
      dtype='object')

In [14]:
# q_col_df is the dataframe with only the question number as the header (Example: Q1, Q2...)
q_cols_df = df.rename(columns= short_cols)

In [15]:
q_cols_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1433 entries, 0 to 1432
Data columns (total 37 columns):
Q1     1433 non-null int64
Q2     1146 non-null object
Q3     1146 non-null float64
Q4     263 non-null float64
Q5     1146 non-null object
Q6     1146 non-null object
Q7     1146 non-null object
Q8     1146 non-null object
Q9     1146 non-null object
Q10    1146 non-null object
Q11    1146 non-null object
Q12    1146 non-null object
Q13    287 non-null object
Q14    287 non-null object
Q15    144 non-null object
Q16    287 non-null object
Q17    287 non-null object
Q18    1264 non-null object
Q19    1433 non-null object
Q20    1433 non-null object
Q21    1344 non-null object
Q22    657 non-null object
Q23    1433 non-null object
Q24    1433 non-null object
Q25    1433 non-null object
Q26    568 non-null object
Q27    1433 non-null object
Q28    711 non-null object
Q29    1433 non-null int64
Q30    1433 non-null int64
Q31    1433 non-null object
Q32    1433 non-null object
Q33    

In [16]:
raw_df = df.rename(columns= long_cols)

In [18]:
# raw df is the data frame with questions as columns
raw_df.head(1)

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,Do you think that discussing a mental health disorder with your employer would have negative consequences?,Would you feel comfortable discussing a mental health disorder with your coworkers?,...,"If so, what condition(s) were you diagnosed with?",Have you ever sought treatment for a mental health issue from a mental health professional?,What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,No,No,I don't know,No,Maybe,...,"Anxiety Disorder (Generalized, Social, Phobia,...",0,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes


In [19]:
# create a for loop to add the question number to the question label (Example: Q1- Are you self-employed?)
q_names_cols = {}
for i in range(0,len(df.columns)):
    q_names_cols[df.columns[i]] =   f"Q{i+1}- {df.columns[i]}"

In [21]:
# q_names_cols

In [23]:
# Create dataframe the the question name and question label 
q_names_df = df.rename(columns= q_names_cols)

In [24]:
#qnames df is the data frame with the question number and the question label
q_names_df.head(1)

,Q1- Are you self-employed?,Q2- How many employees does your company or organization have?,Q3- Is your employer primarily a tech company/organization?,Q4- Is your primary role within your company related to tech/IT?,Q5- Does your employer provide mental health benefits as part of healthcare coverage?,"Q6- Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Q7- Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Q8- Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,Q9- Do you think that discussing a mental health disorder with your employer would have negative consequences?,Q10- Would you feel comfortable discussing a mental health disorder with your coworkers?,...,"Q28- If so, what condition(s) were you diagnosed with?",Q29- Have you ever sought treatment for a mental health issue from a mental health professional?,Q30- What is your age?,Q31- What is your gender?,Q32- What country do you live in?,Q33- What US state or territory do you live in?,Q34- What country do you work in?,Q35- What US state or territory do you work in?,Q36- Which of the following best describes your work position?,Q37- Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,No,No,I don't know,No,Maybe,...,"Anxiety Disorder (Generalized, Social, Phobia,...",0,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes


#### <span style="color:green">q_names_df.columns</span>

In [25]:
q_names_df.columns

Index(['Q1- Are you self-employed?',
       'Q2- How many employees does your company or organization have?',
       'Q3- Is your employer primarily a tech company/organization?',
       'Q4- Is your primary role within your company related to tech/IT?',
       'Q5- Does your employer provide mental health benefits as part of healthcare coverage?',
       'Q6- Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
       'Q7- Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
       'Q8- Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
       'Q9- Do you think that discussing a mental health disorder with your employer would have negative consequences?',
       'Q10- Would you feel comfortable discussing a mental health disorder with your coworkers?',
 

#### <span style="color:deeppink">VARIABLES</span>

In [26]:
self_employed_count = q_names_df[q_names_df['Q1- Are you self-employed?'] == 1][['Q1- Are you self-employed?']].count()
employed_count = q_names_df[q_names_df['Q1- Are you self-employed?'] == 0][['Q1- Are you self-employed?']].count()
total_survey_workers_count = employed_count + self_employed_count
tech_workers_count = q_cols_df[q_cols_df['Q4'] == 1.0]['Q4'].count()

In [27]:
print(self_employed_count[0], employed_count[0], total_survey_workers_count[0])

287 1146 1433


In [28]:
# Count of tech + non tech workers based on answers to Q4[0.0 or 1.0]
print('Tech or not: ', q_cols_df[(q_cols_df['Q4'] == 0.0) | (q_cols_df['Q4'] == 1.0)]['Q4'].count())
print('Tech: ', q_cols_df[q_cols_df['Q4'] == 1.0]['Q4'].count() )
print('non Tech: ', q_cols_df[q_cols_df['Q4'] == 0.0]['Q4'].count() )


Tech or not:  263
Tech:  248
non Tech:  15


#### <span style="color:yellow">Q4: Is Employee Role in tech? +  Cleanup answers to Q: [19,20,21,29]</span>

In [54]:
# tech_workers_df

In [29]:
# Q4: Is Employee Role in tech?
tech_workers_df = pd.DataFrame(q_cols_df[q_cols_df['Q4'] == 1.0])

tech_workers_df['Q4'] = tech_workers_df['Q4'].replace({1.0: "Yes"})

tech_workers_df['Q20'] = tech_workers_df['Q20'].replace({
    "Yes, I think they would": "Yes",
    "Yes, they do": "Yes",
    "No, I don't think they would": "No",
    "No, they do not": "No"
    })

tech_workers_df['Q21'] = tech_workers_df['Q21'].replace({
    "Maybe/Not sure": "Maybe",
    "Yes, I observed": "Yes",
    "Yes, I experienced": "Yes",
    })

tech_workers_df['Q19'] = tech_workers_df['Q19'].replace({
    "Yes, I think it would": "Yes",
    "Yes, it has": "Yes",
    "No, I don't think it would": "No",
    "No, it has not": "No"
    })

tech_workers_df['Q29'] = tech_workers_df['Q29'].replace({
    1: "Yes",
    0: "No"
    })

# tech_workers_df['Q19'].value_counts().index


In [30]:
tech_workers_df = tech_workers_df[['Q31','Q4', 'Q27', 'Q29', 'Q23', 'Q24', 'Q25', 'Q8']]

In [32]:
tech_workers_df.head(3)

,Q31,Q4,Q27,Q29,Q23,Q24,Q25,Q8
4,Female,Yes,Yes,Yes,Yes,Yes,Yes,No
8,Female,Yes,Yes,Yes,Yes,Yes,Yes,I don't know
11,Male,Yes,Yes,Yes,Yes,Yes,Yes,I don't know


#### <span style="color:green">tech_workers grouped by 'Q31': Gender</span>

In [42]:
# Tech Workers  grouped by genders  =======================
tech_gender_group = tech_workers_df.groupby('Q31').count()['Q4']

In [43]:
tech_gender_group

Q31
Female     60
Male      182
Other       6
Name: Q4, dtype: int64

#### <span style="color:lime">tech workers diagnosed by MH pro</span>

In [44]:
# Count of Tech workers diagnosed  =======================================================================
diagnosed_tech_workers = tech_workers_df[tech_workers_df['Q27'] == 'Yes'].groupby('Q31').count()['Q27']

In [71]:
diagnosed_tech_workers

Q31
Female    40
Male      82
Other      5
Name: Q27, dtype: int64

#### <span style="color:yellow">Summary Table: Tech Workers + Diagnosed Tech Workers</span>

In [46]:
tech_and_diagnosed_df = pd.DataFrame({
    'Tech Workers Count': tech_gender_group,
    'Tech Workers %': round((tech_gender_group/tech_gender_group.sum()) * 100,1),
    'Diagnosed Tech Workers': diagnosed_tech_workers,
    'Diagnosed %': round((diagnosed_tech_workers/(tech_gender_group)) * 100,1),
    
})

In [47]:
tech_and_diagnosed_df.sort_values('Tech Workers Count', ascending=False, inplace=True)

In [41]:
tech_and_diagnosed_df

,Tech Workers Count,Tech Workers %,Diagnosed Tech Workers,Diagnosed %
Q31,,,,
Male,182,73.4,82,45.1
Female,60,24.2,40,66.7
Other,6,2.4,5,83.3


#### <span style="color:yellow">Summary Table: Sought Treatment IF anonymity</span>

In [69]:
believes_anonymity_will_be_provided = tech_workers_df[(tech_workers_df['Q27'] == 'Yes') & (tech_workers_df['Q8'] == 'Yes')].groupby('Q31').count()['Q8']  #['Q27']



In [70]:
believes_anonymity_will_be_provided

Q31
Female    10
Male      22
Other      2
Name: Q8, dtype: int64

In [75]:
# 'Q29': Sought Treatment?
tech_workers_df['Q29'].value_counts()

Yes    143
No     105
Name: Q29, dtype: int64

In [80]:
believes_anonymity_sought_treatment = tech_workers_df[(tech_workers_df['Q27'] == 'Yes') & (tech_workers_df['Q8'] == 'Yes') & (tech_workers_df['Q29'] == 'Yes')].groupby('Q31').count()['Q29']  #.count()[]

In [81]:
believes_anonymity_sought_treatment

Q31
Female     9
Male      21
Other      2
Name: Q29, dtype: int64

In [82]:
believes_anonymity_sought_treatment_df = pd.DataFrame({
    'Tech Workers Count': tech_gender_group,
    'Diagnosed Tech Workers': diagnosed_tech_workers,
    'Believes anonymity will be provided': believes_anonymity_will_be_provided,
    'Believes anonymity & sought  treatment': believes_anonymity_sought_treatment
})

In [87]:
believes_anonymity_sought_treatment_df

,Tech Workers Count,Diagnosed Tech Workers,Believes anonymity will be provided,Believes anonymity & sought treatment
Q31,,,,
Female,60,40,10,9
Male,182,82,22,21
Other,6,5,2,2


#### <span style="color:yellow">DOES THE BELIEVE IN HAVING ANONYMITY IF TREATMENT SOUGHT, INCREASE THE CHANCES OF WORKERS SEEKING TREATMENT
</span>

In [132]:
#DIAGNOSED
diagnosed_count = tech_workers_df[(tech_workers_df['Q27'] == 'Yes')]['Q27'].count()
diagnosed_count

127

In [133]:
#DIAGNOSED AND SOUGHT TREATMENT
diagnosed_sought_treamtment_count = tech_workers_df[(tech_workers_df['Q27'] == 'Yes') & (tech_workers_df['Q29'] == "Yes")]['Q29'].count()
diagnosed_sought_treamtment_count

118

In [160]:
#OF THOSE DIAGNOSED, WHAT PERCENTAGE SOUGHT TREATMENT?
diangonsed_sought_treatment_percent = diagnosed_sought_treamtment_count/diagnosed_count
round(diangonsed_sought_treatment_percent*100,1)

92.9

In [152]:
# OF THOSE DIAGNOSED, HOW MANY BELIEVE WILL HAVE ANONYMITY IF THEY SEEK TREATMENT? 
diagnosed_anonymity_count = tech_workers_df[(tech_workers_df['Q27'] == 'Yes') & (tech_workers_df['Q8'] == "Yes")]['Q8'].count()
diagnosed_anonymity_count

34

In [153]:
# OF THOSE DIAGNOSED WHO BELIVE WILL HAVE ANONYMITY,  HOW MANY SOUGHT TREATMENT?
diagnosed_anonymity_sought_treamtment_count = tech_workers_df[(tech_workers_df['Q27'] == 'Yes')& (tech_workers_df['Q8'] == "Yes") & (tech_workers_df['Q29'] == "Yes")]['Q29'].count()
diagnosed_anonymity_sought_treamtment_count


32

In [157]:
#OF THOSE DIAGNOSED THAT BELIEVE WILL HAVE ANONYMITY, WHAT PERCENTAGE SOUGHT TREATMENT?

diagnosed_anonymity_sought_treamtment_percent = diagnosed_anonymity_sought_treamtment_count / diagnosed_anonymity_count
round(diagnosed_anonymity_sought_treamtment_percent * 100, 1)

94.1

In [161]:
# DIAGNOSED BELIEVES NO ANONYMITY
tech_workers_df[(tech_workers_df['Q27'] == 'Yes') & (tech_workers_df['Q8'] == "No")].count()['Q8']  #['Q27']


4

In [162]:
# DIAGNOSED BELIEVES NO ANONYMITY SOUGHT TREATMENT
tech_workers_df[(tech_workers_df['Q27'] == 'Yes') & (tech_workers_df['Q8'] == "No") & (tech_workers_df['Q29'] == 'Yes') ].count()['Q8']  #['Q27']


3

In [176]:
# DIAGNOSED DOES NOT KNOW IF WILL HAVE ANONYMITY
dianosed_notsure_anonymity = tech_workers_df[(tech_workers_df['Q27'] == 'Yes') & (tech_workers_df['Q8'] == "I don't know")]['Q8'].count()
dianosed_notsure_anonymity

89

In [180]:
# DIAGNOSED DOES NOT KNOW IF WILL HAVE ANONYMITY AND SOUGHT TREATMENT
dianosed_notsure_if_anonymity_sought_treatment = tech_workers_df[(tech_workers_df['Q27'] == 'Yes') & (tech_workers_df['Q8'] == "I don't know") & (tech_workers_df['Q29'] == 'Yes') ]['Q29'].count()
dianosed_notsure_if_anonymity_sought_treatment



83

In [181]:
# OF THOSE DIAGNOSED THAT ARE NOT  SURE IF WILL HAVE ANONYMITY, WHAT PERCENTAGE SOUGHT TREATMENT
dianosed_notsure_if_anonymity_sought_treatment_percent =  dianosed_notsure_anonymity_sought_treatment / dianosed_notsure_anonymity
round(dianosed_notsure_if_anonymity_sought_treatment_percent * 100 ,2)


93.26

In [163]:
tech_workers_df['Q8'].value_counts()

I don't know    163
Yes              71
No               14
Name: Q8, dtype: int64

In [184]:
# Tech Workers that sought treatment wether or not they were diagnosed

tech_workers_df['Q29'].value_counts()

Yes    143
No     105
Name: Q29, dtype: int64

In [185]:
# Tech Workers that sought treatment wether or not they were diagnosed percentage

143/248

0.5766129032258065